# Data Project 2

Harrison Listen, Victoria Feist, Adam Cook

In [1]:
# https://www.kaggle.com/datasets/thedevastator/the-ultimate-netflix-tv-shows-and-movies-dataset

In [2]:
import pandas as pd
import csv
import os
import pymongo

# Data Manipulation and Cleaning

In [3]:
# Reading in the data into pandas dataframes
movie_by_year = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/Best Movie by Year Netflix.csv', header=0, index_col=0)
movies = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/Best Movies Netflix.csv', header=0, index_col=0)
show_by_year = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/Best Show by Year Netflix.csv', header=0, index_col=0)
shows = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/Best Shows Netflix.csv', header=0, index_col=0)
credits = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/raw_credits.csv', header=0, index_col=0)
titles = pd.read_csv('/Users/adam/Desktop/Data Science Systems/Final Project/raw_titles.csv', header=0, index_col=0)

In [5]:
# Note that movies is just a more comprehensive version of movies_by_year it looks like.
# So I don't think it is necessary to use movie_by_year or shows_by_year
movies.head()

,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,MAIN_GENRE,MAIN_PRODUCTION
index,,,,,,,
0,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB
1,Inception,2010,8.8,2268288,148,scifi,GB
2,Forrest Gump,1994,8.8,1994599,142,drama,US
3,Anbe Sivam,2003,8.7,20595,160,comedy,IN
4,Bo Burnham: Inside,2021,8.7,44074,87,comedy,US


In [7]:
shows.head()

,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,NUMBER_OF_SEASONS,MAIN_GENRE,MAIN_PRODUCTION
index,,,,,,,,
0,Breaking Bad,2008,9.5,1727694,48,5,drama,US
1,Avatar: The Last Airbender,2005,9.3,297336,24,3,scifi,US
2,Our Planet,2019,9.3,41386,50,1,documentary,GB
3,Kota Factory,2019,9.3,66985,42,2,drama,IN
4,The Last Dance,2020,9.1,108321,50,1,documentary,US


In [8]:
credits.head()

,person_id,id,name,character,role
index,,,,,
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [9]:
#Here, I am filtering out the actors and just looking at the directors for each movie so we can add that to the comprehensive data frame
#If we want to go back and add actors, we can, but think directors is easier b/c usually just one or two per movie
titles = titles.rename(columns={"title":"TITLE"})
credits_without_actors = credits[credits['role'] == 'DIRECTOR']
credits_without_actors = credits_without_actors.rename(columns={"name":"Director"})
titles = pd.merge(titles, credits_without_actors[['id', 'Director']], on='id')
titles.head()

,id,TITLE,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,Director
0,tm84618,Taxi Driver,MOVIE,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,Martin Scorsese
1,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,Terry Jones
2,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,Terry Gilliam
3,tm70993,Life of Brian,MOVIE,1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,Terry Jones
4,tm190788,The Exorcist,MOVIE,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,William Friedkin


In [10]:
# Creating a new dataframe with the information we want
# Note that all_movies_shows is the main data frame I prepared for us to work with
movies['TYPE'] = 'MOVIE'
shows['TYPE'] = 'SHOW'
#this below puts all the movies and shows into the same data frame
all_movies_shows = pd.concat([movies, shows])

In [11]:
# Adding age certification, imdb score, and name of the director to all_movies_shows
# Essentially just merged those first two from titles and then merged the director from titles after getting it from credits above
all_movies_shows = pd.merge(all_movies_shows, titles[['TITLE','age_certification']], on='TITLE')
all_movies_shows = pd.merge(all_movies_shows, titles[['TITLE','imdb_score']], on='TITLE')
all_movies_shows = pd.merge(all_movies_shows, titles[['TITLE', 'Director']], on='TITLE')
all_movies_shows
#print(all_movies_shows[all_movies_shows['TITLE'] == "The Last Dance"])

,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,MAIN_GENRE,MAIN_PRODUCTION,TYPE,NUMBER_OF_SEASONS,age_certification,imdb_score,Director
0,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB,MOVIE,NaN,PG,9.0,Keith Scholey
1,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB,MOVIE,NaN,PG,9.0,Jonathan Hughes
2,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB,MOVIE,NaN,PG,9.0,Alastair Fothergill
3,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB,MOVIE,NaN,PG,9.0,Keith Scholey
4,David Attenborough: A Life on Our Planet,2020,9.0,31180,83,documentary,GB,MOVIE,NaN,PG,9.0,Jonathan Hughes
...,...,...,...,...,...,...,...,...,...,...,...,...
789,All of Us Are Dead,2022,7.5,41393,61,action,KR,SHOW,1.0,TV-MA,7.5,JQ Lee
790,All of Us Are Dead,2022,7.5,41393,61,action,KR,SHOW,1.0,TV-MA,7.5,Kim Nam-soo
791,All of Us Are Dead,2022,7.5,41393,61,action,KR,SHOW,1.0,TV-MA,7.5,JQ Lee
792,All of Us Are Dead,2022,7.5,41393,61,action,KR,SHOW,1.0,TV-MA,7.5,Kim Nam-soo


# Setting up MongoDB and writing dataframe and answers to it

In [23]:
df = all_movies_shows.groupby(["TITLE"]).first()

import pymongo
host_name = "localhost"
port = "27017"
conn_str = {f"mongodb://{host_name}:{port}/"}
client = pymongo.MongoClient(conn_str)
db_name = "Netflix_db"

df_dict = df.to_dict()
db = client[db_name]
dataframe=db.dataframe
dataframe.insert_one(df_dict)

_2020 = df[df["RELEASE_YEAR"] == 2020]
movies_2020 = _2020[_2020['TYPE'] == 'MOVIE']
q1 = movies_2020.sort_values(['SCORE'], ascending = False).head(1)
q1 = q1['SCORE']
q1_dict = q1.to_dict()
db = client[db_name]
question1=db.question1
question1.insert_one(q1_dict)

#Question 2:
_2020 = df[df["RELEASE_YEAR"] == 2020]
shows_2020 = _2020[_2020['TYPE'] == 'SHOW']
q2 = shows_2020.sort_values(['NUMBER_OF_SEASONS'], ascending = False).head(1)
q2 = q2['NUMBER_OF_SEASONS']
q2_dict = q2.to_dict()
question2=db.question2
question2.insert_one(q2_dict)

#Question 3:
_2017 = df[df["RELEASE_YEAR"] >= 2017]
movies_2017 = _2017[_2017['TYPE'] == 'MOVIE']
q3 = movies_2017.sort_values(['NUMBER_OF_VOTES'], ascending = False).head(1)
q3 = q3['Director']
q3_dict = q3.to_dict()
question3=db.question3
question3.insert_one(q3_dict)

#Question 4:
_2022 = df[df["RELEASE_YEAR"] == 2022]
movies_2022 = _2022[_2022['TYPE'] == 'MOVIE']
q4 = movies_2022.sort_values(['SCORE'], ascending = False).head(5).reset_index()
q4.index = q4.index.map(str)
q4 = q4['TITLE']
q4_dict = q4.to_dict()
question4=db.question4
question4.insert_one(q4_dict)

#Question 5:
_2021 = df[df["RELEASE_YEAR"] == 2021]
docs_2021 = _2021[_2021['MAIN_GENRE'] == 'drama']
q5 = docs_2021.sort_values(['SCORE'], ascending = False).head(1).reset_index()
q5.index = q5.index.map(str)
q5 = q5['TITLE']
q5_dict = q5.to_dict()
question5=db.question5
question5.insert_one(q5_dict)

#Question 6:
all_shows = df[df['TYPE'] == 'SHOW']
seasons = all_shows.sort_values(["NUMBER_OF_SEASONS"], ascending = False).head(1)
q6 = seasons['NUMBER_OF_SEASONS']
q6_dict = q6.to_dict()
question6=db.question6
question6.insert_one(q6_dict)

#Question 7:
all_movies = df[df['TYPE'] == "MOVIE"]
director_count = df["Director"].value_counts().head(2)
q7_dict = director_count.to_dict()
question7=db.question7
question7.insert_one(q7_dict)

#Question 8:
all_movies = df[df['TYPE'] == "MOVIE"]
q8 = all_movies.sort_values(['DURATION'], ascending = False).head(1)
q8 = q8['DURATION']
q8_dict = q8.to_dict()
question8=db.question8
question8.insert_one(q8_dict)

#Question 9:
df["RELEASE_YEAR"] = df["RELEASE_YEAR"].astype(str)
yearly_count = df["RELEASE_YEAR"].value_counts().head(1)
q9_dict = yearly_count.to_dict()
question9=db.question9
question9.insert_one(q9_dict)

#Question 10:
all_movies = df[df['TYPE'] == "MOVIE"]
q10 = all_movies.sort_values(['SCORE'], ascending = True).head(1)
q10 = q10['SCORE']
q10_dict = q10.to_dict()
question10=db.question10
question10.insert_one(q10_dict)

# Creating Chatbot

In [13]:
import nltk 

from nltk import word_tokenize,sent_tokenize

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

2022-12-16 20:38:46.825425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [14]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)


Training Step: 1999  | total loss: 0.24613 | time: 0.002s
| Adam | epoch: 1000 | loss: 0.24613 - acc: 0.9512 -- iter: 08/14
Training Step: 2000  | total loss: 0.22225 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.22225 - acc: 0.9561 -- iter: 14/14
--
INFO:tensorflow:/Users/adam/Desktop/Data Science Systems/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /Users/adam/Desktop/Data Science Systems/model.tflearn


## Special Instructions:
- asking any valid question that hits the pattern of one of the tags will return the answers to all ten questions
- if you asked question 1, the answer is the first listed, the answer to the sixth question is the sixth listed, and so on.

In [21]:
def chat():
    print("Say hello to start talking with the bot (type quit to stop)\n")
    print("Say \"help me\" to hear what questions I can answer.")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        if inp.lower() == "help me":
            print("Here are the types of questions I can address:\n1. What is the top movie on Netflix of 2020?\n2. How many seasons are in the top show on Netflix of 2020?\n3. Who directed the most-reviewed movie on Netflix of the past 5 years?\n4. What are the 5 highest-rated movies on Netflix of 2022?\n5. What is the best drama on Netflix from 2021?\n6.What show has the most seasons ever on Netflix?\n7. Which director has made the most movies on Netflix?\n8. What is the longest movie on Netflix?\n9. What year had the most movies and shows?\n10. What is the worst movie in this dataset?\n The information I have only reflects the so-called best Netflix movies and tv shows, based on IMBD ratings, and is not reflective of all movies/tv shows to exist.")


        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]

        if result[result_index] > 0.7:
            for tg in data["intents"]:     
                if tg['tag'] in ['Q1']:#tag and tag == 'Q1':
                    print(db.question1.find_one())
                elif tg['tag'] in ['Q2']:#tag and tag == 'Q2':
                    print(db.question2.find_one())
#                elif tg['tag'] in ['Q3']:#tag and tag == 'Q3':#Explanation for why this is commented out below
 #                   print(db.question3.find_one())
                elif tg['tag'] in ['Q4']:#tag and tag == 'Q4':
                    print(db.question4.find_one())
                elif tg['tag'] in ['Q5']:#tag and tag == 'Q5':
                    print(db.question5.find_one())
                elif tg['tag'] in ['Q6']:#tag and tag == 'Q6':
                    print(db.question6.find_one())
  #              elif tg['tag'] in ['Q7']:#tag and tag == 'Q7':#Explanation for why this is commented out below 
   #                 print(db.question7.find_one())
                elif tg['tag'] in ['Q8']:# tag and tag == 'Q8':
                    print(db.question8.find_one())
                elif tg['tag'] in ['Q9']:#tag and tag == 'Q9':
                    print(db.question9.find_one())
                elif tg['tag'] in ['Q10']:#tag and tag == 'Q10':
                    print(db.question10.find_one())
        elif "most movies" in inp:## We had a lot of trouble figuring out what to put in the tag patterns to result in an output for 3 and 7, so we used phrases in the questions to produce the answer to these.
            print(db.question7.find_one())   
        elif "most-reviewed" or "most reviewed" in inp:
            print(db.question3.find_one())
        else:
            print("I didnt get that. Can you try asking one of these 10 questions:\n1. What is the top movie on Netflix of 2020?\n2. How many seasons are in the top show on Netflix of 2020?\n3. Who directed the most-reviewed movie on Netflix of the past 5 years?\n4. What are the 5 highest-rated movies on Netflix of 2022?\n5. What is the best drama on Netflix from 2021?\n6.What show has the most seasons ever on Netflix?\n7. Which director has made the most movies on Netflix?\n8. What is the longest movie on Netflix?\n9. What year had the most movies and shows?\n10. What is the worst movie in this dataset?\n The information I have only reflects the so-called best Netflix movies and tv shows, based on IMBD ratings, and is not reflective of all movies/tv shows to exist.")
chat()

Say hello to start talking with the bot (type quit to stop)

Say "help me" to hear what questions I can answer.


You:  What was the top movie on Netflix of 2020?


{'_id': ObjectId('639d1da4477726e9751856f6'), 'David Attenborough: A Life on Our Planet': 9.0}
{'_id': ObjectId('639d1da4477726e9751856f7'), 'Rise of Empires: Ottoman': 2.0}
{'_id': ObjectId('639d1da4477726e9751856f9'), '0': 'Dasvi', '1': 'Badhaai Do', '2': 'The Tinder Swindler', '3': 'Gangubai Kathiawadi', '4': 'Radhe Shyam'}
{'_id': ObjectId('639d1da4477726e9751856fa'), '0': 'Young Royals'}
{'_id': ObjectId('639d1da5477726e9751856fb'), 'InuYasha': 9.0}
{'_id': ObjectId('639d1da5477726e9751856fd'), 'Once Upon a Time in America': 229}
{'_id': ObjectId('639d1da5477726e9751856fe'), '2019': 39}
{'_id': ObjectId('639d1da5477726e9751856ff'), 'Chup Chup Ke': 6.9}


You:  How many seasons are in the top show of 2020 on Netflix?


{'_id': ObjectId('639d1da4477726e9751856f6'), 'David Attenborough: A Life on Our Planet': 9.0}
{'_id': ObjectId('639d1da4477726e9751856f7'), 'Rise of Empires: Ottoman': 2.0}
{'_id': ObjectId('639d1da4477726e9751856f9'), '0': 'Dasvi', '1': 'Badhaai Do', '2': 'The Tinder Swindler', '3': 'Gangubai Kathiawadi', '4': 'Radhe Shyam'}
{'_id': ObjectId('639d1da4477726e9751856fa'), '0': 'Young Royals'}
{'_id': ObjectId('639d1da5477726e9751856fb'), 'InuYasha': 9.0}
{'_id': ObjectId('639d1da5477726e9751856fd'), 'Once Upon a Time in America': 229}
{'_id': ObjectId('639d1da5477726e9751856fe'), '2019': 39}
{'_id': ObjectId('639d1da5477726e9751856ff'), 'Chup Chup Ke': 6.9}


You:  How many seasons are in the top show of 2020 on Netflix?


{'_id': ObjectId('639d1da4477726e9751856f6'), 'David Attenborough: A Life on Our Planet': 9.0}
{'_id': ObjectId('639d1da4477726e9751856f7'), 'Rise of Empires: Ottoman': 2.0}
{'_id': ObjectId('639d1da4477726e9751856f9'), '0': 'Dasvi', '1': 'Badhaai Do', '2': 'The Tinder Swindler', '3': 'Gangubai Kathiawadi', '4': 'Radhe Shyam'}
{'_id': ObjectId('639d1da4477726e9751856fa'), '0': 'Young Royals'}
{'_id': ObjectId('639d1da5477726e9751856fb'), 'InuYasha': 9.0}
{'_id': ObjectId('639d1da5477726e9751856fd'), 'Once Upon a Time in America': 229}
{'_id': ObjectId('639d1da5477726e9751856fe'), '2019': 39}
{'_id': ObjectId('639d1da5477726e9751856ff'), 'Chup Chup Ke': 6.9}


KeyboardInterrupt: Interrupted by user